In [1]:
import pandas as pd
import geopandas as gpd

In [ ]:
df = pd.read_csv("80305ned_TypedDataSet_06112025_170745.csv", sep=";")

new_df = pd.DataFrame()
new_df["Region Code"] = df["RegioS"]
new_df["Avg. Distance to GP"] = df["AfstandTotHuisartsenpraktijk_1"]
new_df["Avg. Distance to Pharmacy"] = df["AfstandTotApotheek_6"]
new_df["Avg. Distance to Hospital"] = df[["AfstandTotZiekenhuis_7", "AfstandTotZiekenhuis_11"]].min(axis=1)
new_df["Avg. Distance to Supermarket"] = df["AfstandTotGroteSupermarkt_20"]
new_df["Avg. Distance to Primary School"] = df["AfstandTotSchool_56"]
new_df["Avg. Distance to High School"] = df["AfstandTotSchool_60"]
new_df["Avg. Distance to Highway"] = df["AfstandTotOpritHoofdverkeersweg_100"]
new_df["Avg. Distance to Train Station"] = df["AfstandTotTreinstationsTotaal_101"]
new_df["Avg. Distance to Fire Station"] = df["AfstandTotBrandweerkazerne_125"]
print(len(new_df))
new_df = new_df.dropna() # drop municipalities that don't exist anymore
print(len(new_df)) # should be number of municipalities = 342
new_df = new_df.sort_values("Region Code").reset_index(drop=True)
new_df.head()

500
342


,Region Code,Avg. Distance to GP,Avg. Distance to Pharmacy,Avg. Distance to Hospital,Avg. Distance to Supermarket,Avg. Distance to Primary School,Avg. Distance to High School,Avg. Distance to Highway,Avg. Distance to Train Station,Avg. Distance to Fire Station
0,GM0014,0.9,1.1,3.3,0.8,0.8,1.8,1.2,2.7,2.8
1,GM0034,1.1,1.3,4.4,0.8,0.7,1.6,1.7,2.2,3.2
2,GM0037,1.7,1.4,5.6,1.2,1.0,2.9,1.3,17.5,3.9
3,GM0047,1.3,1.9,2.8,1.3,0.9,2.0,1.9,3.1,2.3
4,GM0050,1.8,1.9,2.4,1.3,1.4,2.4,2.2,14.8,2.2


In [ ]:
gdf = gpd.read_file("wijkenbuurten_2023_v2.gpkg", layer="gemeenten")

new_gdf = gpd.GeoDataFrame(geometry=gdf["geometry"])
new_gdf["Region Code"] = gdf["gemeentecode"]
new_gdf["Region Name"] = gdf["gemeentenaam"]
new_gdf = new_gdf.merge(new_df, how="right", on="Region Code") # merge on region code
print(len(new_gdf))
new_gdf = new_gdf.drop_duplicates("Region Code")
print(len(new_gdf)) # should be number of municipalities = 342
new_gdf.head()

423
342


,geometry,Region Code,Region Name,Avg. Distance to GP,Avg. Distance to Pharmacy,Avg. Distance to Hospital,Avg. Distance to Supermarket,Avg. Distance to Primary School,Avg. Distance to High School,Avg. Distance to Highway,Avg. Distance to Train Station,Avg. Distance to Fire Station
0,"MULTIPOLYGON (((245229.7 592570.87, 245250.71 ...",GM0014,Groningen,0.9,1.1,3.3,0.8,0.8,1.8,1.2,2.7,2.8
1,"MULTIPOLYGON (((150087.299 479382.379, 150000....",GM0034,Almere,1.1,1.3,4.4,0.8,0.7,1.6,1.7,2.2,3.2
3,"MULTIPOLYGON (((263763.866 566430.392, 263944....",GM0037,Stadskanaal,1.7,1.4,5.6,1.2,1.0,2.9,1.3,17.5,3.9
4,"MULTIPOLYGON (((256504.591 572265.478, 256541....",GM0047,Veendam,1.3,1.9,2.8,1.3,0.9,2.0,1.9,3.1,2.3
5,"MULTIPOLYGON (((155047.32 474836.246, 155049.0...",GM0050,Zeewolde,1.8,1.9,2.4,1.3,1.4,2.4,2.2,14.8,2.2


In [ ]:
new_gdf.to_file("preprocessed_dataset.json", driver="GeoJSON") # merge csv and shapefile as geojson